BPTT 原理 + 为什么对 (h_t) 求偏导 + 直观解释
---

# 通过时间反向传播（BPTT）原理与推导（从标量链式法则视角）

## 1. BPTT 的核心思想

给定 RNN 结构：

$$
h_t = \tanh(w_x x_t + w_h h_{t-1}),\qquad
y_t = w_y h_t,\qquad
l_t = \frac12 (y_t - \hat{y}_t)^2
$$

总损失为各时间步损失之和：

$$
L = \sum_{t=1}^T l_t
$$

由于某些中间量（如 (h_t)）出现在多个时间步的计算路径中，因此对它们的梯度需要**把所有路径的贡献加起来**：

$$
\frac{\partial L}{\partial h_t}
= \sum_{\text{所有经过 } h_t \text{ 通往 } L \text{ 的路径}}
\frac{\partial L}{\partial h_t}\Big|_{\text{该路径}}
$$

这本质上就是：

> **“哪里有自变量，沿哪条路链式求导，并把所有路径相加。”**

---

## 2. 为什么要对 (h_t) 求偏导？它明明不是参数！

虽然我们不更新隐状态 (h_t)，
但为了对参数（如 (w_x, w_h, w_y)）求梯度，必须使用链式法则：

例如：

$$
\frac{\partial L}{\partial w_x}
= \sum_t
\frac{\partial L}{\partial a_t}
\cdot \frac{\partial a_t}{\partial w_x}
$$

而

$$
\frac{\partial L}{\partial a_t}
= \frac{\partial L}{\partial h_t}
\cdot \frac{\partial h_t}{\partial a_t}
$$

于是 $(\frac{\partial L}{\partial h_t})$ 就是**计算参数梯度所必需的“中转站梯度”**。
它不是用来更新 (h_t)，而是用来继续将梯度往前传播。

总结一句话：

> **对 (h_t) 求偏导不是为了优化 (h_t)，
> 而是为了让梯度能沿着计算图继续传到真正的参数。
> (h_t) 在反向传播中只是“梯度高速公路的中间收费站”。**

---

## 3. 标量 RNN 示例：两步序列，从头算到尾

我们构造一个简单的一维 RNN：

* 参数：
  (w_x = 1,\ w_h = 1,\ w_y = 1)
* 输入：
  (x_1 = 1,\ x_2 = 1)
* 目标输出：
  (\hat y_1=0,\ \hat y_2=0)
* 初始状态：
  (h_0 = 0)

### 3.1 正向传播

第一步：

$$
a_1 = 1,\quad
h_1 = \tanh(1)=0.7616
$$

$$
y_1 = h_1 = 0.7616
$$

$$
l_1 = \tfrac12 y_1^2 \approx 0.2908
$$

第二步：

$$
a_2 = 1 + h_1 = 1.7616,\quad
h_2 = \tanh(1.7616)=0.9427
$$

$$
y_2 = h_2,\quad
l_2 = \tfrac12 y_2^2 \approx 0.4441
$$

总损失：

$$
L = l_1+l_2 \approx 0.7349
$$

---

## 3.2 输出层反向（从 (y_t) 回来）

对平方误差：

$$
\frac{\partial l_t}{\partial y_t} = y_t - \hat y_t
$$

因此：

$$
\frac{\partial l_1}{\partial h_1} = y_1 = 0.7616
$$

$$
\frac{\partial l_2}{\partial h_2} = y_2 = 0.9427
$$

记隐状态的“上游梯度”为：

$$
\delta_t^h := \frac{\partial L}{\partial h_t}
$$

因为 (h_2) 只影响 (l_2)：

$$
\delta_2^h = 0.9427
$$

---

## 3.3 BPTT 的递推：未来的梯度也要回传

因为

$$
h_t = \tanh(a_t),\qquad
a_t = w_x x_t + w_h h_{t-1}
$$

有

$$
\frac{\partial h_t}{\partial a_t} = 1 - h_t^2
$$

对于 (h_2)：

$$
\frac{\partial L}{\partial a_2}
= \delta_2^h \cdot (1-h_2^2)
= 0.9427\cdot 0.1113
= 0.1050
$$

这会进一步传给 (h_1)：

$$
\frac{\partial L}{\partial h_1}\Big|_{\text{来自 } t=2}
= \frac{\partial L}{\partial a_2}\cdot w_h
= 0.1050
$$

因此 (h_1) 的总梯度：

$$
\delta_1^h
= \underbrace{\frac{\partial l_1}{\partial h_1}}_{\text{来自当前}}
$$
$$
 \underbrace{\frac{\partial L}{\partial h_1}|*{\text{来自 } t=2}}
 $$
$$
{\text{来自未来}}
  = 0.7616 + 0.1050
  = 0.8666
$$

这体现了 BPTT 的本质：

> **某个隐状态的梯度 = 本时间步损失的贡献 + 所有未来时间步的贡献。**

---

## 3.4 最终参数梯度 = 时间求和

对 (w_x)：

$$
\frac{\partial L}{\partial w_x}
= \sum_t \frac{\partial L}{\partial a_t}\cdot \frac{\partial a_t}{\partial w_x}
= 0.3639 + 0.1050
= 0.4689
$$

对 (w_h)：

$$
\frac{\partial L}{\partial w_h}
= \sum_t \frac{\partial L}{\partial a_t}\cdot \frac{\partial a_t}{\partial w_h}
= 0 + 0.0799
= 0.0799
$$

对 (w_y)（输出层参数）：

$$
\frac{\partial L}{\partial w_y}
= \sum_t (y_t - \hat y_t) h_t
= 1.4687
$$

---

## 4. BPTT 的统一递推公式（标量版）

隐状态梯度递推：

$$
\delta_t^h
= (y_t - \hat y_t) w_y
$$
$$
 \delta_{t+1}^h \cdot (1 - h_{t+1}^2) w_h
  $$

参数梯度：

$$
\frac{\partial L}{\partial w_x}
= \sum_t \delta_t^h (1-h_t^2)x_t
$$

$$
\frac{\partial L}{\partial w_h}
= \sum_t \delta_t^h (1-h_t^2)h_{t-1}
$$

$$
\frac{\partial L}{\partial w_y}
= \sum_t (y_t - \hat y_t)h_t
$$

这些就是 RNN/BPTT 的基本推导。

---

## 5. 最重要的理解：

### 对 (h_t) 求偏导不是因为要优化它，而是因为链式法则必须经过它

**隐状态不是参数，但它是多条通往损失 L 的路径的“交叉节点”。
为了让梯度继续往前传，我们必须知道它的上游梯度。**

所以：

* **反向传播会对所有中间变量求梯度**（包括 (h_t)、(a_t)、(y_t)）
* **但只有参数的梯度（如 (w_x,w_h,w_y)）会被优化器使用**

一句话总结：

> **对 (h_t) 求偏导是为了“把梯度传过去”，
> 而不是为了“更新 h_t”。
> 它的角色是：链式法则中的中间站。**

